### ST1502 CA1

Name: Jovan Heng Ghim Hong

Class: DAAA/2B/22

Admin No: 2401418

Main Problem Statement/Goal:

Recommend ideas that can help singapore with housing issues

In [44]:
# Basic Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import warnings

warnings.filterwarnings('ignore')

##### Basic Data Exploration

In [71]:
df = pd.read_csv('./datasets/HDB Resale flat prices-2017-2025v2.csv')
df

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,@406,AMK AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New-Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3-ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New-Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,AMK AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,AMK,3-ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 y 05 months,265000.0
...,...,...,...,...,...,...,...,...,...,...,...
202759,2025-02,YISHUN,EXECUTIVE,723,YISHUN ST 71,07 TO 09,146.0,Maisonette,1986,60 years 05 months,818888.0
202760,2025-01,YISHUN,EXECUTIVE,836,YISHUN ST 81,01 TO 03,146.0,Maisonette,1988,62 years 02 months,960000.0
202761,2025-02,YISHUN,EXECUTIVE,824,YISHUN ST 81,01 TO 03,145.0,Apartment,1987,61 years 10 months,868888.0
202762,2025-02,YISHUN,EXECUTIVE,826,YISHUN ST 81,10 TO 12,146.0,Maisonette,1988,62 years,1100888.0


In [21]:
## Looking at unique values
discrete_cols = ['town' , 'flat_type' , 'storey_range' , 'flat_model' , 'block']

for col in discrete_cols:
  print(col)
  print(np.unique(df[col].values))
  print('===========================================')

town
['AMK' 'ANG MO KIO' 'BEDOK' 'BISHAN' 'BUKIT BATOK' 'BUKIT MERAH'
 'BUKIT PANJANG' 'BUKIT TIMAH' 'CCK' 'CENTRAL AREA' 'CHOA CHU KANG'
 'CLEMENTI' 'GEYLANG' 'HOUGANG' 'JURONG EAST' 'JURONG WEST'
 'KALLANG/WHAMPOA' 'MARINE PARADE' 'PASIR RIS' 'PUNGGOL' 'QUEENSTOWN'
 'SEMBAWANG' 'SENG-KANG' 'SENGKANG' 'SERANGOON' 'TAMPINES' 'TOA PAYOH'
 'WOODLANDS' 'YISHUN']
flat_type
['1 ROOM' '2 ROOM' '3 RM' '3 ROOM' '3 ROOMS' '3 ROOMs' '3-ROOM' '4 ROOM'
 '4 ROOMS' '4 ROOMSS' '5 ROOM' '5-RM' 'EXECUTIVE' 'MULTI-GENERATION']
storey_range
['01 -- 03' '01 ---03' '01 TO 03' '01----03' '01@03' '01@@03' '04 -- 06'
 '04 TO 06' '04---06' '07 -- 09' '07 --- 09' '07 TO 09' '07@@09'
 '10 TO 12' '10--12' '10-Dec' '10@12' '13 - 15' '13 TO 15' '16 TO 18'
 '19 -- 21' '19 TO 21' '22 -- 24' '22 --- 24' '22 TO 24' '25 TO 27'
 '28 TO 30' '31 TO 33' '34 TO 36' '37 TO 39' '4-Jun' '40 TO 42' '43 TO 45'
 '46 TO 48' '49 TO 51']
flat_model
['2-room' '3Gen' 'Adjoined flat' 'Apartment' 'DBSS' 'Improved'
 'Improved-Maisonette' 

There are multiple different values that have the same meaning.

<br>
For Example:

  - **AMK** is the same as **Ang Mo Kio**
  - The **multiple variations** of storey range **1 TO 3**
  - The many **mis-spelling** of **3 ROOMS** etc

We can make all of those into one singular **'code'/representation**

In [10]:
## Looking at summaries, etc
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202764 entries, 0 to 202763
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                202764 non-null  object 
 1   town                 202764 non-null  object 
 2   flat_type            202764 non-null  object 
 3   block                202764 non-null  object 
 4   street_name          202764 non-null  object 
 5   storey_range         202764 non-null  object 
 6   floor_area_sqm       202764 non-null  float64
 7   flat_model           202764 non-null  object 
 8   lease_commence_date  202764 non-null  int64  
 9   remaining_lease      202764 non-null  object 
 10  resale_price         202764 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 17.0+ MB


There is no missing data

The datatypes of reslae price and month is wrong

In [20]:
## descriptive summaries
# df['resale_price'] = df['resale_price'].astype(float)
df.describe()

,floor_area_sqm,lease_commence_date
count,202764.000000,202764.000000
mean,96.929510,1996.244718
std,24.030239,14.188254
min,31.000000,1966.000000
25%,82.000000,1985.000000
50%,93.000000,1996.000000
75%,112.000000,2011.000000
max,366.700000,2021.000000


##### Data Cleaning

In [37]:
## The concept behind cleaning is similar for all
def clean_discrete_data(mapping, col , df):
  """
Map contains x : y, where searches for any instance of x in each cell to replace to y
  """
  # Get the data
  data = df[col]
  def change_data(x):
    for keys in mapping.keys():
      if keys in x:
        return mapping[keys]
    return x
  
  data = data.apply(change_data)
  return data

In [41]:
replacements = {
  'town': {
    'AMK': 'ANG MO KIO',
    'CCK': 'CHOA CHU KANG',
    'SENG-KANG': 'SENGKANG'
  },
  'flat_type': {
    '3': '3 ROOM',
    '4': '4 ROOM',
    '5': '5 ROOM'
  },
  'storey_range': {
    **{
      # gives '03': '01 to 03' for i = 03
      f"{i:02}": f"{i - 2:02} to {i:02}"
      for i in range(3 , 52 , 3)
    },
    '10-Dec': '10 to 12', # implied where dec is 12th month
    '4-Jun': '04-06'
  }, 
}

# allows me to still work with original data
cleaned_df = df.copy()

for col , map in replacements.items():
  cleaned_df[col] = clean_discrete_data(map , col , df)

In [63]:
# for block will be abit different will use regex
regex = r"(\d+)([A-Z]?)"

cleaned_df['block'] = cleaned_df['block'].apply(lambda x: re.search(regex , x).group())

In [64]:
# relook at unique values

for col in discrete_cols:
  print(col)
  print(np.unique(cleaned_df[col].values))
  print('===========================================')

town
['ANG MO KIO' 'BEDOK' 'BISHAN' 'BUKIT BATOK' 'BUKIT MERAH' 'BUKIT PANJANG'
 'BUKIT TIMAH' 'CENTRAL AREA' 'CHOA CHU KANG' 'CLEMENTI' 'GEYLANG'
 'HOUGANG' 'JURONG EAST' 'JURONG WEST' 'KALLANG/WHAMPOA' 'MARINE PARADE'
 'PASIR RIS' 'PUNGGOL' 'QUEENSTOWN' 'SEMBAWANG' 'SENGKANG' 'SERANGOON'
 'TAMPINES' 'TOA PAYOH' 'WOODLANDS' 'YISHUN']
flat_type
['1 ROOM' '2 ROOM' '3 ROOM' '4 ROOM' '5 ROOM' 'EXECUTIVE'
 'MULTI-GENERATION']
storey_range
['01 to 03' '04 to 06' '04-06' '07 to 09' '10 to 12' '13 to 15' '16 to 18'
 '19 to 21' '22 to 24' '25 to 27' '28 to 30' '31 to 33' '34 to 36'
 '37 to 39' '40 to 42' '43 to 45' '46 to 48' '49 to 51']
flat_model
['2-room' '3Gen' 'Adjoined flat' 'Apartment' 'DBSS' 'Improved'
 'Improved-Maisonette' 'Maisonette' 'Model -A' 'Model A'
 'Model A-Maisonette' 'Model A2' 'Multi Generation' 'NG' 'New Generation'
 'New-Generation' 'Premium Apartment' 'Premium Apartment Loft'
 'Premium Maisonette' 'Simplified' 'Standard' 'Terrace' 'Type S1'
 'Type S2']
block
['1' '10' 

In [65]:
# Here we can change the dtype
cleaned_df['month'] = pd.to_datetime(cleaned_df['month'] , format='%Y-%m')

In [69]:
cleaned_df['resale_price'] = cleaned_df['resale_price'].astype(float)

ValueError: could not convert string to float: '285-000'

There is an error 😱, suggest there are some badly formatted strings inside of this col

In [76]:
def convert_resale_to_numbers(x):
  try:
    return float(x)
  except:
    # will be triggered when tries to float strings etc
    return x

In [77]:
# Convert to float
cleaned_df['resale_price'] = cleaned_df['resale_price'].apply(convert_resale_to_numbers)

In [79]:
# select those that are not classed as numbers
cleaned_df[cleaned_df['resale_price'].apply(lambda x: not (isinstance(x , float)))]

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
88750,2021-01-01,KALLANG/WHAMPOA,3 ROOM,11,UPP BOON KENG RD,04 to 06,65.0,Improved,1974,52 years,285-000


In [80]:
# can just replace it
cleaned_df['resale_price'] = cleaned_df['resale_price'].apply(lambda x: 285000 if isinstance(x , str) else x)

In [82]:
cleaned_df[cleaned_df['resale_price'].apply(lambda x: not (isinstance(x , float)))].size

0

The size shows 0, suggesting that everything is a float

##### Exporting Data

In [85]:
cleaned_df.to_excel('./outputs/cleaned_data.xlsx')